In [1]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks/3_NeuralNetworks/autoencoder.ipynb

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [7]:
num_features = 784

learning_rate = 0.01
training_steps = 20000
batch_size = 256
display_step = 1000

num_hidden_1 = 128
num_hidden_2 = 64

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

x_train, x_test = x_train / 255., x_test / 255.

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(10000).batch(batch_size).prefetch(1)


test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = train_data.repeat().batch(batch_size).prefetch(1)

In [24]:
random_normal = tf.initializers.RandomNormal()

weights = {
    'encoder_h1' : tf.Variable( random_normal( [num_features, num_hidden_1] ) ),
    'encoder_h2' : tf.Variable( random_normal( [num_hidden_1, num_hidden_2 ] ) ),
    'decoder_h2' : tf.Variable( random_normal( [num_hidden_2, num_hidden_1] ) ),
    'decoder_h1' : tf.Variable( random_normal( [num_hidden_1, num_features] ) )
}

biases = {
    'encoder_b1' : tf.Variable( random_normal( [num_hidden_1] ) ),
    'encoder_b2' : tf.Variable( random_normal( [num_hidden_2] ) ),
    'decoder_b1' : tf.Variable( random_normal( [num_hidden_1] ) ),
    'decoder_b2' : tf.Variable( random_normal( [num_features] ) )
}

In [25]:
def encoder(x):

    layer_1 = tf.nn.sigmoid( tf.matmul( x, weights['encoder_h1'] ) + biases['encoder_b1'] )

    layer_2 = tf.nn.sigmoid( tf.matmul( layer_1, weights['encoder_h2'] ) + biases['encoder_b2'] )

    return layer_2

def decoder(x):

    layer_1 = tf.nn.sigmoid( tf.matmul( x, weights['decoder_h2'] ) + biases['decoder_b1'] )

    layer_2 = tf.nn.sigmoid( tf.matmul( layer_1, weights['decoder_h1'] ) + biases['decoder_b2'] )

    return layer_2

In [26]:
def mean_square(reconstructed, original):
    
    return tf.reduce_mean( tf.pow( original - reconstructed, 2 ) )

optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [27]:
def run_optimization(x):

    with tf.GradientTape() as g:

        reconstructed = decoder(encoder(x))
        loss = mean_square(reconstructed, x)
    
    trainable_variables = weights.values() + biases.values()

    gradients = g.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss


In [28]:

# Run training for the given number of steps.
for step, (batch_x, _) in enumerate(train_data.take(training_steps + 1)):
    
    # Run the optimization.
    loss = run_optimization(batch_x)
    
    if step % display_step == 0:
        print("step: %i, loss: %f" % (step, loss))

InternalError: Blas GEMM launch failed : a.shape=(256, 784), b.shape=(784, 128), m=256, n=128, k=784 [Op:MatMul] name: MatMul/